In [1]:
import sys

# TO CHANGE
BASEDIR = "/home/dzigen/Desktop/PersonalAI/Personal-AI"
# TO CHNAGE

sys.path.insert(0, BASEDIR)

from time import time
from dataclasses import dataclass
import numpy as np
from typing import Dict, List

from src.neo4j_functions import Neo4jConnection
from src.embedding_functions import EmbeddingsDatabaseConnection, EmbeddingsDatabaseConnectionConfig, EmbedderModel, EmbedderModelConfig, VectorDBConnectionConfig
from src.knowledge_graph_model import KnowledgeGraphModel

from src.qa_pipeline.knowledge_retriever.AStarTripletsRetriever import Neo4jGraphDriver
from src.utils.data_structs import create_id_for_node_pair, NodeType
from src.qa_pipeline.knowledge_retriever.cache import KeyValueStore, KeyValueStoreConfig

/home/dzigen/Desktop/PersonalAI/Personal-AI/pai_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
kg_model = KnowledgeGraphModel(
    graph_db=Neo4jConnection(uri='bolt://localhost:7687', user='neo4j', pwd='password', db_name='diaasq2'),
    embeddings_db=EmbeddingsDatabaseConnection(EmbeddingsDatabaseConnectionConfig(
        embedder_config=EmbedderModelConfig(model_name_or_path='../../models/intfloat/multilingual-e5-small'),
        nodes_db_config=VectorDBConnectionConfig(
            '../../data/graph_structures/vectorized_nodes/v10/densedb', 'vectorized_nodes', is_exist=True, need_to_clear=False
        ),
        triplets_db_config=VectorDBConnectionConfig(
            '../../data/graph_structures/vectorized_triplets/v6/densedb', 'vectorized_triplets', is_exist=True, need_to_clear=False
        )
    ))
)

No sentence-transformers model found with name ../../models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


In [13]:
cache = KeyValueStore()

In [14]:
astar_metrics = AStarMetrics(kg_model, cache=cache, accepted_node_types=f'["{NodeType.object.value}","{NodeType.hyper.value}","{NodeType.episodic.value}"]')

In [6]:
data = kg_model.embeddings_db.vectordbs['nodes'].collection.get(include=['embeddings'], limit=10)

In [7]:
data['ids']

['4:d6769385-179e-400c-982b-6502bc94d278:0',
 '4:d6769385-179e-400c-982b-6502bc94d278:1',
 '4:d6769385-179e-400c-982b-6502bc94d278:2',
 '4:d6769385-179e-400c-982b-6502bc94d278:3',
 '4:d6769385-179e-400c-982b-6502bc94d278:4',
 '4:d6769385-179e-400c-982b-6502bc94d278:5',
 '4:d6769385-179e-400c-982b-6502bc94d278:6',
 '4:d6769385-179e-400c-982b-6502bc94d278:7',
 '4:d6769385-179e-400c-982b-6502bc94d278:8',
 '4:d6769385-179e-400c-982b-6502bc94d278:9']

In [16]:
s_time = time()
astar_metrics.weighted_short_path('4:d6769385-179e-400c-982b-6502bc94d278:2', '4:d6769385-179e-400c-982b-6502bc94d278:1')
e_time = time()
print(e_time - s_time)

calculating
passed nodes:  22
calculating
0.16015958786010742


In [18]:
s_time = time()
astar_metrics.precomputed_dist('4:d6769385-179e-400c-982b-6502bc94d278:9', '4:d6769385-179e-400c-982b-6502bc94d278:1')
e_time = time()
print(e_time - s_time)

exists
0.0006651878356933594
